## Model Test

Finally, we want to evaluate our model on our test set. To do so, we'll uniformly sample 10 clips from each test video, to produce a tensor of [10 x 3 x (8 or 32) x 112 x 112] that we call our prediction on. This will give us the clip-level accuracy for batches of 10 at time. We also want to calculate the video-level accuracy which we can get by simply averaging the clip-level accuracy.

In [1]:
import torch
import os
import numpy as np
import argparse

from model import VideoLearner
from dataset import VideoRecord, VideoDataset

In [2]:
model_path = os.path.join('.')
model_name = 'test_015'
num_frames = 32
data_root = os.path.join('ActivityNet', 'Crawler', 'Kinetics')
video_dir = os.path.join(data_root, 'data2')
test_txt = os.path.join(data_root,'test1.txt')

In [3]:
data = VideoDataset(video_dir, 
                    train_split_file=test_txt, 
                    test_split_file=test_txt,
                    batch_size=4,
                    sample_length=num_frames)

model = VideoLearner(data, num_classes=57)
model.load(model_name=model_name, model_dir=model_path)

ActivityNet/Crawler/Kinetics/test1.txt
Loading r2plus1d_34_32_ig65m model


Using cache found in /home/shkim/.cache/torch/hub/moabitcoin_ig65m-pytorch_master


In [4]:
ret = model.evaluate()

/home/shkim/anaconda3/envs/AIC/lib/python3.7/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


score:  0.1101


KeyError: 'video_pred'

In [5]:
ds = data.test_ds

In [6]:
ds[0]

(tensor([[[[[ 0.7704,  0.7725,  0.7692,  ..., -0.6547, -0.9794, -0.8633],
            [ 0.9091,  0.9607,  1.0123,  ..., -1.2449, -0.8422, -0.9301],
            [ 0.1685,  0.1685,  2.0998,  ..., -1.2569, -0.8930, -0.8540],
            ...,
            [-1.7920, -1.7920, -1.8092,  ...,  0.4643,  0.5146,  0.5146],
            [-1.7920, -1.7920, -1.8092,  ...,  0.2997,  0.3738,  0.3738],
            [-1.7920, -1.7920, -1.8092,  ...,  0.3061,  0.4039,  0.4448]],
 
           [[ 0.7704,  0.7725,  0.7692,  ..., -0.6383, -0.9724, -0.8684],
            [ 0.9091,  0.9607,  1.0123,  ..., -1.2395, -0.8046, -0.9311],
            [ 0.1685,  0.1685,  2.0998,  ..., -1.2920, -0.8953, -0.8872],
            ...,
            [-1.7920, -1.7920, -1.8092,  ...,  0.4643,  0.5146,  0.5146],
            [-1.7920, -1.7920, -1.8092,  ...,  0.2997,  0.3738,  0.3738],
            [-1.7920, -1.7920, -1.8092,  ...,  0.3061,  0.4039,  0.4448]],
 
           [[ 0.7107,  0.7359,  0.7016,  ..., -0.8435, -0.9767, -0.8974]

In [10]:
model.model.eval()

with torch.no_grad():
    inputs, label = ds[130]
    inputs = inputs.to('cuda', non_blocking=True)
    
    outputs = model.model(inputs)
    outputs = outputs.cpu().numpy()
    
    print(outputs.sum(axis=0).argmax())
    print(label)

44
16


In [ ]:
import pandas as pd
